In [1]:
# !dir

In [2]:
# !pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [4]:
from bs4 import BeautifulSoup as bs
from hdfs import InsecureClient
import pandas as pd
import folium
from datetime import datetime

In [5]:
# Chrome 옵션 설정
options = Options()
options.binary_location = '/usr/local/bin/chrome-headless-shell' # 크롬 브라우저 경로
# options.add_argument('--headless')
options.add_argument('--no-sandbox') # 실행 가드? 해제
options.add_argument('--disable-dev-shm-usage') # 공유 메모리 비활성화
options.add_argument('--disable-gpu') # GPU 비활성화
options.add_argument('--window-size=1920,1080') # 가상 화면크기 설정
# ChromeDriver 설정
# service = Service('/usr/local/bin/chromedriver')
service = webdriver.ChromeService() # executable_path='/usr/local/bin/chromedriver'
driver = webdriver.Chrome(service=service, options=options)

In [6]:
driver.get('https://www.starbucks.co.kr/index.do')
print(driver.title)

hover_el = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a')
menu_opt = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li.gnb_sub_ttl > a')

# 마우스 호버 액션
time.sleep(3)
actions = ActionChains(driver)
actions.move_to_element(hover_el).perform() # perform == run
print('[[호버 동작]]')
# 대기시간
time.sleep(1.5)
print('[[매장메뉴 선택]]')
menu_opt.click()

print('[[페이지 이동 중]]')
time.sleep(6.5)
driver.find_element(By.CSS_SELECTOR, '.loca_search > h3 > a').click()
time.sleep(0.5)
driver.find_element(By.CSS_SELECTOR, '.sido_arae_box > li:nth-child(6) > a').click()
time.sleep(0.5)
driver.find_element(By.CSS_SELECTOR, '.gugun_arae_box > li:nth-child(1) > a').click()

Starbucks Korea
[[호버 동작]]
[[매장메뉴 선택]]
[[페이지 이동 중]]


In [7]:
# 현재 페이지 정보 가져오기
star_list = []
html = driver.page_source
soup = bs(html, 'html.parser')
contents = soup.select_one('.quickSearchResultBoxSidoGugun')
listnames = contents.select('li')
for listname in listnames:
    star_list.append([listname['data-name'], listname['data-lat'], listname['data-long'], listname.select_one('p').text, listname.select_one('i')['class'][0]])

driver.quit()


In [8]:
columns = ['name', 'lat', 'lon', 'addr', 'stype']
starbucks = pd.DataFrame(star_list, columns=columns)

In [9]:
starbucks.head(10)

,name,lat,lon,addr,stype


In [10]:
# starbucks.groupby('stype').describe()
starbucks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    0 non-null      object
 1   lat     0 non-null      object
 2   lon     0 non-null      object
 3   addr    0 non-null      object
 4   stype   0 non-null      object
dtypes: object(5)
memory usage: 132.0+ bytes


In [11]:
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
print(timestamp)

20250102075216


In [18]:
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
csv_path = f'starbucks_{timestamp}.csv'
csv_file = starbucks.to_csv(csv_path, index=True)
print(f'변환 완료: {csv_path}')
path = f'./resultData/{csv_path}'

try:
    path = f'/user/hadoop/{csv_path}'
    hdfs = InsecureClient('http://bdata.iptime.org:9870', user='hadoop')
    hdfs.upload(path, csv_path)
    print('hdfs 업로드 완료')
except Exception as e:
    print(f'hdfs 업로드 실패 - {e}')


변환 완료: starbucks_20250102075417.csv
hdfs 업로드 실패 - HTTPConnectionPool(host='datanode1', port=9864): Max retries exceeded with url: /webhdfs/v1/user/hadoop/starbucks_20250102075417.csv?op=CREATE&user.name=hadoop&namenoderpcaddress=namenode:9000&createflag=&createparent=true&overwrite=false&user.name=hadoop (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdc0ef4e250>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [13]:
try:
    print(120 / 0 * 33)
except Exception as e:
    print(f'오류 발생 - {e}')

오류 발생 - division by zero


In [14]:
data = starbucks.copy()
coordinates = data[['lat', 'lon']].to_numpy()
coordinates

array([], shape=(0, 2), dtype=object)

In [15]:
place = data[['name']].to_numpy()
place

array([], shape=(0, 1), dtype=object)

In [16]:
cn_lat, cn_lon = coordinates[4]
print(cn_lat, cn_lon)
m = folium.Map(location=[cn_lat, cn_lon], zoom_start=11)

for i, (lat, lon) in enumerate(coordinates):
    # print(lat, type(lat))
    folium.CircleMarker(
        location=[lat, lon],
        radius=10,
        color='blue',
        fill=True,
        fill_color='skyblue',
        fill_opacity=0.5,
        tooltip=place[i]
    ).add_to(m)
    # 라벨 추가 (Marker로 위치 조정)
    folium.Marker(
        location=[float(lat) + 0.001, float(lon)],  # 위도 약간 증가
        icon=folium.DivIcon(html=f'<div style="font-size: 9px; color: black; background-color: white; width:50px;white-space: nowrap; overflow: hidden;text-overflow: ellipsis;">{place[i]}</div>')
    ).add_to(m)
# 확인
m

IndexError: index 4 is out of bounds for axis 0 with size 0